<a href="https://colab.research.google.com/github/DiegoLegarda/EntrenamientoClima/blob/main/RedClima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
import pickle
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [12]:
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import os

# Listar archivos en una carpeta específica
ruta_carpeta = '/content/drive/MyDrive/Colab Notebooks'
archivos = os.listdir(ruta_carpeta)
print(archivos)

['DatosEntrenamiento.csv', 'DatosEntrenamiento.gsheet', 'RedClima.ipynb']


In [14]:
import pandas as pd

# Definir la ruta al archivo
ruta_archivo = '/content/drive/MyDrive/Colab Notebooks/DatosEntrenamiento.csv'

# Cargar el archivo en un DataFrame
data = pd.read_csv(ruta_archivo, decimal=',')

# Ver las primeras filas
print(data.head())

# 1. Cargar y explorar los datos

data['fecha_hora'] = pd.to_datetime(data['fecha_hora'])
data['temperatura'] = pd.to_numeric(data['temperatura'], errors='coerce')
data['humedad'] = pd.to_numeric(data['humedad'], errors='coerce')
print(data)

      fecha_hora  temperatura    humedad
0  20230101T0000    12.270246  80.826390
1  20230101T0100    12.340245  79.099785
2  20230101T0200    11.840245  80.655174
3  20230101T0300    11.550245  81.831300
4  20230101T0400    11.260245  81.179360
               fecha_hora  temperatura    humedad
0     2023-01-01 00:00:00    12.270246  80.826390
1     2023-01-01 01:00:00    12.340245  79.099785
2     2023-01-01 02:00:00    11.840245  80.655174
3     2023-01-01 03:00:00    11.550245  81.831300
4     2023-01-01 04:00:00    11.260245  81.179360
...                   ...          ...        ...
15691 2024-10-15 19:00:00          NaN        NaN
15692 2024-10-15 20:00:00          NaN        NaN
15693 2024-10-15 21:00:00          NaN        NaN
15694 2024-10-15 22:00:00          NaN        NaN
15695 2024-10-15 23:00:00          NaN        NaN

[15696 rows x 3 columns]


In [15]:
# 2. Limpieza de datos
data_clean = data.dropna()
data_clean = data_clean.drop_duplicates()

In [16]:
# 3. Ingeniería de características
data_clean['hora'] = data_clean['fecha_hora'].dt.hour
data_clean['dia'] = data_clean['fecha_hora'].dt.day
data_clean['mes'] = data_clean['fecha_hora'].dt.month
data_clean['dia_semana'] = data_clean['fecha_hora'].dt.dayofweek

In [23]:
# 4. Etiquetado automático
def clasificar_clima(row):
    if row['temperatura'] > 30 and row['humedad'] < 40:
        return 1  # Alto Riesgo
    elif row['temperatura'] < 20 and row['humedad'] > 70:
        return 0  # Bajo Riesgo
    else:
        return 2  # Riesgo Medio

data_clean['clima'] = data_clean.apply(clasificar_clima, axis=1)
data_final = data_clean[data_clean['clima'] != 2]

def clasificar_clima(row):
    if row['temperatura'] > 30 and row['humedad'] < 40:
        return 'caluroso'
    elif row['temperatura'] < 15 and row['humedad'] > 80:
        return 'frío y húmedo'
    elif row['temperatura'] > 25 and row['humedad'] > 50:
        return 'templado y húmedo'
    else:
        return 'templado'

data_clean['clima_tipo'] = data_clean.apply(clasificar_clima, axis=1)

In [28]:
# 5. Preparación para el modelo
X = data_final[['temperatura', 'humedad', 'hora', 'dia', 'mes', 'dia_semana']]
y_riesgo = data_clean['clima']
y_clima_tipo = data_clean['clima_tipo']


# Verificar las dimensiones de X, y_riesgo, y_clima_tipo
print("Dimensión de X:", X.shape)
print("Dimensión de y_riesgo:", y_riesgo.shape)
print("Dimensión de y_clima_tipo:", y_clima_tipo.shape)

# Combina X y las etiquetas en un DataFrame para asegurar consistencia
data_combined = pd.concat([X, y_riesgo, y_clima_tipo], axis=1)

# Elimina filas con valores NaN en cualquiera de las columnas
data_combined = data_combined.dropna()

# Separa de nuevo las características y las etiquetas
X = data_combined[['temperatura', 'humedad', 'hora', 'dia', 'mes', 'dia_semana']]
y_riesgo = data_combined['clima']
y_clima_tipo = data_combined['clima_tipo']

# Ahora revisa nuevamente las dimensiones para verificar que coincidan
print("Dimensión de X después de limpiar:", X.shape)
print("Dimensión de y_riesgo después de limpiar:", y_riesgo.shape)
print("Dimensión de y_clima_tipo después de limpiar:", y_clima_tipo.shape)



# Dividimos los datos de entrada (X) y las dos variables de salida (y_riesgo y y_clima_tipo)
X_train, X_test, y_riesgo_train, y_riesgo_test = train_test_split(
    X, y_riesgo, test_size=0.2, random_state=42
)
_, _, y_clima_tipo_train, y_clima_tipo_test = train_test_split(
    X, y_clima_tipo, test_size=0.2, random_state=42
)

# Escalamos solo los datos de entrada (X_train y X_test)
scaler = StandardScaler()

# Ajustamos el escalador con los datos de entrenamiento y luego transformamos
X_train_scaled = scaler.fit_transform(X_train)

# Transformamos los datos de prueba con el escalador ajustado
X_test_scaled = scaler.transform(X_test)

# Verificamos los tamaños
print("Tamaño de X_train_scaled:", X_train_scaled.shape)
print("Tamaño de X_test_scaled:", X_test_scaled.shape)
print("Tamaño de y_riesgo_train:", y_riesgo_train.shape)
print("Tamaño de y_clima_tipo_train:", y_clima_tipo_train.shape)

Dimensión de X: (10145, 6)
Dimensión de y_riesgo: (15384,)
Dimensión de y_clima_tipo: (15384,)
Dimensión de X después de limpiar: (10145, 6)
Dimensión de y_riesgo después de limpiar: (10145,)
Dimensión de y_clima_tipo después de limpiar: (10145,)
Tamaño de X_train_scaled: (8116, 6)
Tamaño de X_test_scaled: (2029, 6)
Tamaño de y_riesgo_train: (8116,)
Tamaño de y_clima_tipo_train: (8116,)


In [29]:
# Guardar el scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


In [30]:
# Entrenamiento del modelo para clasificar el riesgo
model_riesgo = RandomForestClassifier(n_estimators=100, random_state=42)
model_riesgo.fit(X_train, y_riesgo_train)

# Predicciones para riesgo
y_riesgo_pred = model_riesgo.predict(X_test)

# Evaluación del modelo de riesgo
print("Reporte de Clasificación - Riesgo:")
print(classification_report(y_riesgo_test, y_riesgo_pred))
print("Matriz de Confusión - Riesgo:")
print(confusion_matrix(y_riesgo_test, y_riesgo_pred))

Reporte de Clasificación - Riesgo:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2004
           1       1.00      1.00      1.00        25

    accuracy                           1.00      2029
   macro avg       1.00      1.00      1.00      2029
weighted avg       1.00      1.00      1.00      2029

Matriz de Confusión - Riesgo:
[[2004    0]
 [   0   25]]


In [31]:
# Entrenamiento del modelo para clasificar el tipo de clima
model_clima = RandomForestClassifier(n_estimators=100, random_state=42)
model_clima.fit(X_train, y_clima_tipo_train)

# Predicciones para tipo de clima
y_clima_tipo_pred = model_clima.predict(X_test)

# Evaluación del modelo de clima
print("Reporte de Clasificación - Tipo de Clima:")
print(classification_report(y_clima_tipo_test, y_clima_tipo_pred))
print("Matriz de Confusión - Tipo de Clima:")
print(confusion_matrix(y_clima_tipo_test, y_clima_tipo_pred))

Reporte de Clasificación - Tipo de Clima:
               precision    recall  f1-score   support

     caluroso       1.00      1.00      1.00        25
frío y húmedo       1.00      1.00      1.00      1224
     templado       1.00      1.00      1.00       780

     accuracy                           1.00      2029
    macro avg       1.00      1.00      1.00      2029
 weighted avg       1.00      1.00      1.00      2029

Matriz de Confusión - Tipo de Clima:
[[  25    0    0]
 [   0 1224    0]
 [   0    0  780]]


In [34]:
# modelo entrenado llamado 'modelo_riesgo'
joblib.dump(model_riesgo, 'modelo_riesgo.pkl')

# modelo para el tipo de clima
joblib.dump(model_clima, 'modelo_clima_tipo.pkl')

['modelo_clima_tipo.pkl']